# Redbuss project

In [129]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import time
import csv
#1
driver = webdriver.Chrome()

url_dic={
    'Kerala':'https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile',
    'telangana':'https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile',
    'Bihar ': 'https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc',
    'West bengal':'https://www.redbus.in/online-booking/west-bengal-transport-corporation',
    'Sikkim ': 'https://www.redbus.in/online-booking/sikkim-nationalised-transport-snt',
    'Meghalaya': 'https://www.redbus.in/online-booking/meghalaya-transport-corporation-mtc',
    'Assam ':'https://www.redbus.in/online-booking/astc',
    'Punjab': 'https://www.redbus.in/online-booking/pepsu-punjab',
    'Himachal Pradesh': 'https://www.redbus.in/online-booking/himachal-pradesh-tourism-development-corporation-hptdc',
    'Andhra Pradesh':'https://www.redbus.in/online-booking/apsrtc'
    
    
}

bus_data = []
try:
    for state, url in url_dic.items():
        driver.get(url)
        time.sleep(5)

        wait = WebDriverWait(driver, 10)
        page = 1

        all_routes = []
 #2       
        while True:
            print(f"Processing page: {page}")
            try:
                nested_div = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]')))
                route_list = nested_div.find_elements(By.CLASS_NAME, 'route_link')  

                for route in route_list:
                    try:
                        link_element = route.find_element(By.TAG_NAME, 'a')
                        route_name = link_element.get_attribute('title')  
                        xroute_link = link_element.get_attribute('href')
                        all_routes.append((state, route_name, route_link))
                    except Exception as e:
                        print(f"Error extracting route data: {e}")

                page_buttons = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".DC_117_pageTabs")))

                if page >= len(page_buttons): 
                    print("Reached the last page.")
                    break

                next_button = page_buttons[page]
                driver.execute_script("arguments[0].click();", next_button)
                
                page+=1
                time.sleep(2)  

            except Exception as e:
                print(f"Error navigating data on page {page}: {e}")
                break

        for state, route_name, route_link in all_routes:
            try:
                driver.get(route_link)
                time.sleep(3)

                try:
                    view_buses_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".button")))
                    view_buses_button.click() 
                except Exception as e:
                            print(f"no busses for this route {e}")

                bus_details = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))
                
                for bus in bus_details:
                    try:
                        bus_name = bus.find_element(By.CLASS_NAME, "travels").text
                        bus_type = bus.find_element(By.CLASS_NAME, "bus-type").text
                        departure_time = bus.find_element(By.CLASS_NAME, "dp-time").text
                        duration = bus.find_element(By.CLASS_NAME, "dur").text
                        reaching_time = bus.find_element(By.CLASS_NAME, "bp-time").text
                        
                        try:
                            star_rating = bus.find_element(By.CSS_SELECTOR, ".rating-sec .rating span").text
                        except Exception:
                            star_rating  = "N/A"
                        
                        fare = bus.find_element(By.CLASS_NAME, "fare").text.split()[-1]  # Extract numeric fare
       
                        try:
                            seats_available = bus.find_element(By.CLASS_NAME, "seat-left").text.split()[0]
                        except Exception:
                            seats_available = "N/A"

                        bus_data.append({
                            "state": state,
                            "route_name": route_name,
                            "route_link": route_link,
                            "busname": bus_name,
                            "bustype": bus_type,
                            "departing_time": departure_time,
                            "duration": duration,
                            "reaching_time": reaching_time,
                            "star_rating": star_rating,
                            "price": fare,
                            "seats_available": seats_available
                            })
                    except Exception as e:
                        print(f"Error extracting bus details: {e}")

            except Exception as e:
                print(f"Error processing route link: {e}")


    with open("final_busdata.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=[
            "state", "route_name", "route_link", "busname", "bustype",
            "departing_time", "duration", "reaching_time", "star_rating", "price", "seats_available"
        ])
        writer.writeheader()
        writer.writerows(bus_data)
finally:
    driver.quit()

Processing page: 1
Processing page: 2
Reached the last page.
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Processing page: 1
Processing page: 2
Processing page: 3
Processing page: 4
Reached the last page.
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: list index out of range
Error extracting bus details: 

In [131]:
pd.read_csv('final_busdata.csv')

,state,route_name,route_link,busname,bustype,departing_time,duration,reaching_time,star_rating,price,seats_available
0,Kerala,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,Kalpaka Travels 1,A/C Sleeper (2+1),23:59,06h 01m,06:00,4.3,900.0,2
1,Kerala,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,UFO Bus,Volvo Multi-Axle B9R A/c Semi Sleeper (2+2),23:55,05h 35m,05:30,3.8,700.0,5
2,Kerala,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,Greenline kerala,A/C Seater Push Back (2+2),01:30,04h 30m,06:00,4.4,500.0,26
3,Kerala,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,Sonia Travels,A/C Seater/Sleeper (2+1),23:30,04h 59m,04:29,4.4,700.0,6
4,Kerala,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 581,Super Express Non AC Seater Air Bus (2+2),10:00,07h 45m,17:45,4.4,667.0,8
...,...,...,...,...,...,...,...,...,...,...,...
1432,Andhra Pradesh,Hyderabad to Kanigiri,https://www.redbus.in/bus-tickets/hyderabad-to...,V Kaveri Travels,Non A/C Seater / Sleeper (2+1),21:30,09h 30m,07:00,4.3,560.0,30
1433,Andhra Pradesh,Hyderabad to Kanigiri,https://www.redbus.in/bus-tickets/hyderabad-to...,V Kaveri Travels,A/C Sleeper (2+1),21:30,09h 30m,07:00,4.1,700.0,22
1434,Andhra Pradesh,Hyderabad to Kanigiri,https://www.redbus.in/bus-tickets/hyderabad-to...,Raja Buses,NON AC Seater / Sleeper 2+1,22:00,08h 55m,06:55,4.1,500.0,31
1435,Andhra Pradesh,Hyderabad to Kanigiri,https://www.redbus.in/bus-tickets/hyderabad-to...,JOSH BUS,Non A/C Seater / Sleeper (2+1),20:45,10h 05m,06:50,4.4,500.0,31


In [3]:
df=pd.read_csv('final_busdata.csv')